In [1]:
my_input = "day22_my_input.txt"
test_input = "day22_test_input.txt"

In [93]:
from operator import add

class VirusCarrier:
    def __init__(self, init_position):
        self._position = init_position
        self._absolute_direction = "UP"
        
        self._directions_map = {
            "UP": (-1, 0),
            "DOWN": (1, 0),
            "LEFT": (0, -1),
            "RIGHT": (0, 1)
        }
        
        self._left_turn_map = {
            "UP": "LEFT",
            "DOWN": "RIGHT",
            "LEFT": "DOWN",
            "RIGHT": "UP"
        }
        
        self._right_turn_map = {
            "UP": "RIGHT",
            "DOWN": "LEFT",
            "LEFT": "UP",
            "RIGHT": "DOWN"
        }
        
        self._reverse_map = {
            "UP": "DOWN",
            "DOWN": "UP",
            "LEFT": "RIGHT",
            "RIGHT": "LEFT"
        }
        
    @property
    def position(self):
        return self._position
        
    def turn_left(self):
        self._absolute_direction = self._left_turn_map[self._absolute_direction]
    
    def turn_right(self):
        self._absolute_direction = self._right_turn_map[self._absolute_direction]
        
    def reverse(self):
        self._absolute_direction = self._reverse_map[self._absolute_direction]
    
    def move(self):
        self._position = tuple(map(add,
                                   self._position,
                                   self._directions_map[self._absolute_direction]))

## Part 1

In [82]:
class InfectionSimulation:
    def __init__(self, data_file, bursts=5):
        self._infected_nodes, self._start_node = self._load_grid(data_file)
        self._infected_by_carrier = 0
        self._simulate_infection(bursts)
        
    @property
    def infected_by_carrier(self):
        return self._infected_by_carrier
    
    def _load_grid(self, data_file):
        infected_nodes = set()
        
        with open(data_file) as f:
            for row, line in enumerate(f):
                for column, char in enumerate(line):
                    if char == "#":
                        infected_node = (row, column)
                        infected_nodes.add(infected_node)
        
        return infected_nodes, (row // 2, column // 2)
    
    def _simulate_infection(self, bursts):
        carrier = VirusCarrier(self._start_node)
        
        for _ in range(bursts):
            current_node = carrier.position
            
            if current_node in self._infected_nodes:
                self._infected_nodes.remove(current_node)
                carrier.turn_right()
            else:
                self._infected_nodes.add(current_node)
                self._infected_by_carrier += 1
                carrier.turn_left()
            
            carrier.move()

In [83]:
assert(InfectionSimulation(test_input, bursts=70).infected_by_carrier == 41)
assert(InfectionSimulation(test_input, bursts=10000).infected_by_carrier == 5587)
print("Tests passed")

Tests passed


In [84]:
InfectionSimulation(my_input, bursts=10000).infected_by_carrier

5552

## Part 2

In [94]:
from enum import Enum, auto
from collections import defaultdict

class State(Enum):
    CLEAN = auto()
    WEAKENED =  auto()
    INFECTED = auto()
    FLAGGED = auto()

class InfectionSimulation:
    def __init__(self, data_file, bursts=5):
        self._grid, self._start_node = self._process_grid(data_file)
        self._infected_by_carrier = 0
        
        self._state_transition_map = {
            State.CLEAN: State.WEAKENED,
            State.WEAKENED: State.INFECTED,
            State.INFECTED: State.FLAGGED,
            State.FLAGGED: State.CLEAN
        }
        
        self._simulate_infection(bursts)
        
    @property
    def infected_by_carrier(self):
        return self._infected_by_carrier
    
    def _process_grid(self, data_file):
        grid = defaultdict(lambda: State.CLEAN)
        
        with open(data_file) as f:
            for row, line in enumerate(f):
                for column, char in enumerate(line):
                    if char == "#":
                        infected_node = (row, column)
                        grid[infected_node] = State.INFECTED
        
        return grid, (row // 2, column // 2)
    
    def _simulate_infection(self, bursts):
        carrier = VirusCarrier(self._start_node)
        
        for burst in range(bursts):
            if not burst % 500000:
                print(burst)
                
            current_node = carrier.position
            current_node_state = self._grid[current_node]
            
            if current_node_state == State.CLEAN:
                carrier.turn_left()
            elif current_node_state == State.WEAKENED:
                pass #don't move in this case
            elif current_node_state == State.INFECTED:
                carrier.turn_right()
            elif current_node_state == State.FLAGGED:
                carrier.reverse()
            else:
                raise NotImplementedError()
            
            self._grid[current_node] = self._transition_state(current_node_state)
            
            carrier.move()
    
    def _transition_state(self, original_state):
        new_state = self._state_transition_map[original_state]
        
        if new_state == State.INFECTED:
            self._infected_by_carrier += 1
        
        return new_state

In [98]:
assert(InfectionSimulation(test_input, bursts=100).infected_by_carrier == 26)
assert(InfectionSimulation(test_input, bursts=10000000).infected_by_carrier == 2511944)
print("Tests passed")

0
0
500000
1000000
1500000
2000000
2500000
3000000
3500000
4000000
4500000
5000000
5500000
6000000
6500000
7000000
7500000
8000000
8500000
9000000
9500000
Tests passed


In [97]:
InfectionSimulation(my_input, bursts=10000000).infected_by_carrier

0
500000
1000000
1500000
2000000
2500000
3000000
3500000
4000000
4500000
5000000
5500000
6000000
6500000
7000000
7500000
8000000
8500000
9000000
9500000


2511527